# HTTP Application routing
The HTTP application routing solution makes it easy to access applications that are deployed to your Azure Kubernetes Service (AKS) cluster. When the solution's enabled, it configures an Ingress controller in your AKS cluster. As applications are deployed, the solution also creates publicly accessible DNS names for application endpoints.

When the add-on is enabled, it creates a DNS Zone in your subscription.

The add-on deploys two components: a Kubernetes Ingress controller and an External-DNS controller.

* Ingress controller: The Ingress controller is exposed to the internet by using a Kubernetes service of type LoadBalancer. The Ingress controller watches and implements Kubernetes Ingress resources, which creates routes to application endpoints.
* External-DNS controller: Watches for Kubernetes Ingress resources and creates DNS A records in the cluster-specific DNS zone.

## Prerequisites
Install Azure [CLI](https://docs.microsoft.com/en-us/cli/azure/install-azure-cli)
* Login to Azure CLI using the command **az login**
* set subscription with **az account set --subscription "name"**

## Deploy HTTP Routing- CLI
The HTTP application routing add-on can be enabled with the Azure CLI when deploying an AKS cluster. To do so, use the az aks create command with the --enable-addons argument.

In [ ]:
az aks create --resource-group myResourceGroup --name myAKSCluster --enable-addons http_application_routing

You can also enable HTTP routing on an existing AKS cluster using the az aks enable-addons command. To enable HTTP routing on an existing cluster, add the --addons parameter and specify http_application_routing as shown in the following example:

In [ ]:
az aks enable-addons --resource-group myResourceGroup --name myAKSCluster --addons http_application_routing

After the cluster is deployed or updated, use the az aks show command to retrieve the **DNS zone name**.

In [ ]:
az aks show --resource-group myResourceGroup --name myAKSCluster --query addonProfiles.httpApplicationRouting.config.HTTPApplicationRoutingZoneName -o table

# Use HTTP routing
The HTTP application routing solution may only be triggered on Ingress resources that are annotated as follows:

In [ ]:
annotations:
  kubernetes.io/ingress.class: addon-http-application-routing

Add the annotations to the following service and open a file named http-application-routing.yaml:

In [ ]:
apiVersion: networking.k8s.io/v1
kind: Ingress
metadata:
  name: eclwatch-ingress
  annotations:
    kubernetes.io/ingress.class: addon-http-application-routing
    kubernetes.io/ingress.class: nginx
spec:
  rules:
    - host: <DNS_ZONE_NAME>
      http:
        paths:
        - path: /
          pathType: Prefix
          backend:
           service:
             name: eclwatch
             port: 
               number: 8010

Use the kubectl apply command to create the resources.

In [ ]:
kubectl apply -f http-application-routing.yaml

# Remove HTTP routing
The HTTP routing solution can be removed using the Azure CLI. To do so run the following command, substituting your AKS cluster and resource group name.

In [ ]:
az aks disable-addons --addons http_application_routing --name myAKSCluster --resource-group myResourceGroup --no-wait

When the HTTP application routing add-on is disabled, some Kubernetes resources may remain in the cluster. These resources include configMaps and secrets, and are created in the kube-system namespace. To maintain a clean cluster, you may want to remove these resources.

Look for addon-http-application-routing resources using the following kubectl get commands:

In [ ]:
kubectl get deployments 
kubectl get services 
kubectl get configmaps 
kubectl get secrets 

To delete resources, use the kubectl delete command. Specify the resource type, resource name, and namespace. The following example deletes one of the previous configmaps:

In [ ]:
kubectl delete configmaps addon-http-application-routing-nginx-configuration

# Troubleshoot
Use the kubectl logs command to view the application logs for the External-DNS application. The logs should confirm that an A and TXT DNS record were created successfully.

In [ ]:
kubectl logs -f deploy/addon-http-application-routing-external-dns -n kube-system

Use the kubectl logs command to view the application logs for the Nginx Ingress controller. The logs should confirm the CREATE of an Ingress resource and the reload of the controller. All HTTP activity is logged.

In [ ]:
kubectl logs -f deploy/addon-http-application-routing-nginx-ingress-controller

# Cleanup
Remove the associated Kubernetes objects created in this article using kubectl delete.

In [ ]:
kubectl delete -f http-application-routing.yaml